In [1]:
import os
os.system('wget https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Annotations_Train_mscoco.zip')
os.system('unzip v2_Annotations_Train_mscoco.zip')
os.system('rm v2_Annotations_Train_mscoco.zip')


0

In [11]:
import json
import pickle
import operator
from collections import Counter

frequent_ans={} # dict to store frequency of each answer
ans_file1=open("/content/v2_mscoco_train2014_annotations.json") # Open the answer file
data_ans1=json.load(ans_file1) #Load the json file
count=0

"""### Iterating over file and finding the frequency of each answer

"""
count=0
frequent_ans={}
for i in data_ans1['annotations']: # for the frequent answert f
    count+=1
    if i['multiple_choice_answer'] in frequent_ans:
      frequent_ans[i['multiple_choice_answer']]+=1
    else:
      frequent_ans[i['multiple_choice_answer']]=0

# Closing file
ans_file1.close()

"""### Sorting the Dictonary

"""

frequent_ans_sorted = dict(sorted(frequent_ans.items(),key=operator.itemgetter(1),reverse=True))

"""### Finding most frequent 3000 answers

"""

count_ans = Counter(frequent_ans_sorted)
most_common_ans = dict(count_ans.most_common(3000))
print(most_common_ans)

"""### Storing the answer in file

"""

try:
    frequent_answers_file = open('frequent_answers', 'wb')
    pickle.dump(most_common_ans, frequent_answers_file)
    frequent_answers_file.close()
  
except:
    print("Something went wrong")

{'yes': 84977, 'no': 82515, '1': 12539, '2': 12214, 'white': 8915, '3': 6535, 'blue': 5454, 'red': 5200, 'black': 5065, '0': 4976, '4': 4117, 'brown': 3813, 'green': 3749, 'yellow': 2791, '5': 2366, 'gray': 2112, 'nothing': 1813, 'right': 1765, 'frisbee': 1640, 'baseball': 1596, 'left': 1564, 'none': 1562, 'tennis': 1501, '6': 1454, 'wood': 1448, 'orange': 1424, 'bathroom': 1229, 'pizza': 1202, 'pink': 1201, 'kitchen': 1092, '10': 980, '7': 937, 'cat': 932, '8': 910, 'dog': 889, 'water': 887, 'man': 884, 'skateboarding': 883, 'grass': 878, 'skiing': 865, 'kite': 792, 'silver': 772, 'black and white': 765, 'surfing': 761, 'horse': 707, 'living room': 701, 'skateboard': 700, 'phone': 696, 'snow': 640, 'wii': 635, 'giraffe': 635, 'woman': 631, 'standing': 626, 'surfboard': 621, 'eating': 606, 'cake': 600, 'food': 598, 'apple': 585, 'sunny': 583, 'broccoli': 571, 'table': 563, 'hat': 556, 'stop': 555, 'purple': 545, 'laptop': 543, 'elephant': 538, '12': 526, 'sheep': 519, '9': 515, 'snowbo

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [4]:
os.system('wget http://nlp.stanford.edu/data/glove.6B.zip')
os.system('unzip glove*.zip')
os.system('rm glove*.zip')

0

In [6]:
def read_data(file_name):
    with open(file_name,'r') as f:
        word_vocab = set() # not using list to avoid duplicate entry
        word2vector = {}
        for line in f:
            line_ = line.strip() #Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word2vector[words_Vec[0]] = np.array(words_Vec[1:],dtype=float)
    print("Total Words in DataSet:",len(word_vocab))
    return word_vocab,word2vector

In [7]:
vocab, w2v = read_data('/content/glove.6B.50d.txt')

Total Words in DataSet: 400000


In [8]:
def cos_sim(u,v):
    """
    u: vector of 1st word
    v: vector of 2nd Word
    """
    numerator_ = u.dot(v)
    denominator_= np.sqrt(np.sum(np.square(u))) * np.sqrt(np.sum(np.square(v)))
    return numerator_/denominator_

In [ ]:
!pip install transformers

In [ ]:
!pip install sentence_transformers

In [31]:
import json
import pickle
from transformers import pipeline
import h5py
import numpy
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

classifier = SentenceTransformer('sentence-transformers/distilbert-base-nli-max-tokens')



In [32]:
def TextFeatureExtractor(word):
    return classifier.encode(word)

In [ ]:
freq_ans_vec={}
for i in most_common_ans:
  print(i)
  i_vec=TextFeatureExtractor(i)
  freq_ans_vec[i]=i_vec
print(freq_ans_vec)

In [37]:
non_freq_to_freq={}
count=0
for i in frequent_ans:
  if (i not in most_common_ans) and (i not in non_freq_to_freq) :
    min=1
    ans=""
    i_vec=TextFeatureExtractor(i)
    for j in most_common_ans:
      temp=cos_sim(i_vec,freq_ans_vec[j])
      if(temp<min):
        min=temp
        ans=j
    non_freq_to_freq[i]=ans
print(non_freq_to_freq)

{'contrail': 'giraffe and zebra', 'white and purple': 'roast beef', 'document': 'feeding giraffe', 'screen projection': 'shearing sheep', 'outlook web app': 'no flowers', 'near tree': 'zebra and giraffe', 'homeless': 'playing wii', 'cross legged': '100 year party ct', 'automatic': 'feeding giraffe', 'wmatacom': 'macaroni and cheese', 'sierra nevada': 'fishing boat', 'on cliff': 'macaroni and cheese', '6:10': 'no parking', 'from': 'macaroni and cheese', 'on top of hill': 'ketchup and mustard', 'melted': 'catching frisbee', 'sweet potato fries': "can't tell", 'long ways': 'french fries', 'peace way': 'halloween', 'everts air cargo': 'potato salad', 'dutchsimba': 'no cat', 'cargo plane': 'happy birthday', 'floor lamp': 'catching frisbee', 'one in corner': 'zebra and giraffe', 'guatemala': 'feeding giraffe', 'md': 'catching frisbee', 'petting sheep': 'miami', 'del taco': 'mother and child', 'florist': 'feeding giraffe', 'special': 'zebra and giraffe', 'blue yellow': 'halloween', 'post-its'

In [38]:
try:
    non_frequent_mapping_file = open('non_frequent_mapping', 'wb')
    pickle.dump(non_freq_to_freq, non_frequent_mapping_file)
    non_frequent_mapping_file.close()
  
except:
    print("Something went wrong")